---
# 환경 설정
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers==4.3.0
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# pip install transformers-interpret

In [5]:
import wandb

wandb.login()
wandb.init()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: glad415. Use `wandb login --relogin` to force relogin


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [8]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

bertmodel = AutoModel.from_pretrained("/content/drive/MyDrive/aiffel/AIFFELTON/TAPT_Model_Save")

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/aiffel/AIFFELTON/TAPT_Model_Save and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---
# 학습용 데이터셋 전처리
---

In [11]:
train_data = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/3_element_dataset_modifying/Final_Train_dataset_0920.csv')
# del train_data['Unnamed: 0']
train_data

,review,taste,quantity,delivery
0,로제는 역시 배떡 배달비도 싸고 좋습니다,0,0,0
1,배달비가 싸서 좋네요,0,0,0
2,매장보다 4000원비싸게 구매했는데 배달비따로받음 결국6000원비싸게배달함 배달은 ...,0,0,0
3,배달비 비싼건 빼고 괜춘함,0,0,0
4,업무가 많이서 배달이 안되면 어플을 내리건가 고객한테 결제 할 수 있도록 그대로 해...,0,0,0
...,...,...,...,...
44175,먹을 때도 조금 냄새가 별로긴 했는데 치킨 식고 나니까 누린내가 너무 나서 간장 양...,1,1,1
44176,성신여대점보다 간이 약한 느낌 양도 많고 배달도 빨리 오셔서 감사한데 맛은 그저 그...,1,1,1
44177,맛과 양은 양심껏 드립니다. 근데 배달에 130분 걸리는 건 양심이 과하게 없네요,1,1,1
44178,점심시간인지 배달은 너무 늦게 오네요.. 반반때문인지 양은 적네요..튀긴껍질은 너무...,1,1,1


In [12]:
valid_data = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/3_element_dataset_modifying/Fianl_Valid_dataset_0915.csv')
valid_data

,review,taste,quantity,delivery
0,처음시켜봤는데 맛있고 빠른배달에 서비스 스파게티도 잘억었습니다,1,1,1
1,맛 좋고 빠르고 양많고,1,1,1
2,맛있고 배달 빠르고 가성비 좋아요 서비스도 굳,1,1,1
3,아주 맛있게 잘 먹었습니다 양도 많고 따뜻하게 배달되어서 좋았습니다,1,1,1
4,맛 훌륭함 양 적당함 배달 매우 빠름 p s 매번 리뷰 이벤트를 까먹는 것 같은 기...,1,1,1
...,...,...,...,...
1094,언제 먹어도 후회가 없어요 이벤트도 감사합니다,0,0,0
1095,순삭이에요 애들점심으로 줬더니 내꺼로 남을게 없네요ㅠ 다음에 또 주문할께요,0,0,0
1096,피자를 오랜만에 먹었네요 다른 곳보다 저렴하게 판매하셔서 감사합니다,0,0,0
1097,불편함없이 잘먹었습니다,0,0,0


In [13]:
import numpy as np 

total_data_text = list(train_data['review'])
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  31.82485287460389
문장길이 최대 :  300
문장길이 표준편차 :  26.266715651157654
pad_sequences maxlen :  84
전체 문장의 0.9588727931190584%가 maxlen 설정값 이내에 포함됩니다. 


---
# 모델
---

In [7]:
class BERTDataset(Dataset): 
  def __init__(self, dataset, bert_tokenizer, max_len):
    
    self.sentences = [bert_tokenizer(i, truncation=True, return_token_type_ids=True, padding="max_length",
                                     max_length = max_len, return_tensors = 'pt') for i in dataset['review']]
    self.max_len = max_len
    self.taste = [np.int64(i) for i in dataset['taste']]
    self.quantity = [np.int64(i) for i in dataset['quantity']]
    self.delivery = [np.int64(i) for i in dataset['delivery']]
  
  def __getitem__(self,i):
    self.input_ids = self.sentences[i]['input_ids']
    self.attention_mask_token = self.sentences[i]['attention_mask']
    self.token_type_ids = self.sentences[i]['token_type_ids']

    return self.input_ids, self.attention_mask_token, self.token_type_ids, self.taste[i], self.quantity[i], self.delivery[i]

  def __len__(self):
    return (len(self.taste))

In [8]:
# 하이퍼파라미터 세팅
max_len = 84
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate = 6e-6


In [16]:
data_train = BERTDataset(train_data, tokenizer, max_len)
data_valid = BERTDataset(valid_data, tokenizer, max_len)

In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, batch_size=batch_size, num_workers=3, drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(data_valid, shuffle=False, batch_size=batch_size, num_workers=3, drop_last=True)

In [18]:
wandb.init(project = 'klue_bert_TAPT_before',
                 entity="glad415",
                 config = {
                     "hidden_size" : 768,
                     "max_len" : max_len,
                     "batch_size" : batch_size,
                     "learning_rate" : learning_rate,
                     "epochs" : num_epochs,
                     "optimizer" : 'adamW',
                     "loss" : "BCEloss",
                 })

In [9]:
from transformers.modeling_utils import PreTrainedModel

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 ################# 클래스 수 조정 ##################
                 num_classes=1,   
                 dr_rate=None,
                 params=None,):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.taste = nn.Linear(hidden_size , num_classes)
        self.quantity = nn.Linear(hidden_size , num_classes)
        self.delivery = nn.Linear(hidden_size , num_classes)

        self.dropout = nn.Dropout(p=dr_rate)

        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask, token_type_ids):

        out = self.bert(input_ids = input_ids.long(), token_type_ids = token_type_ids.long(), attention_mask = attention_mask.to(device))
        out = self.dropout(out.pooler_output)

        taste_out = self.taste(out)
        quantity_out = self.quantity(out)
        delivery_out = self.delivery(out)   

        return self.sigmoid(taste_out), self.sigmoid(quantity_out), self.sigmoid(delivery_out)
    
    ############################# Frezee #############################
    # def freeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = False
    
    # def unfreeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = True
    ##################################################################

In [20]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.2).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.BCELoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

print(warmup_step)
print(t_total)

276
2760


In [21]:
# def custom_micro_f1(pred, label):
#   label_list = ['맛','양','배달']
#   label_indices = list(range(len(label_list)))
#   return f1_score(pred, label, labels= label_indices, average = 'micro')

In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    total_f1 = 0.0
    for batch_id, (input_ids, attention_mask_token, token_type_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(train_dataloader)):
        
        optimizer.zero_grad()
        
        input_ids = input_ids.long().to(device)
        input_ids = input_ids.squeeze(1)

        token_type_ids = token_type_ids.long().to(device)
        token_type_ids = token_type_ids.squeeze(1)
        
        attention_mask_token= attention_mask_token.squeeze(1)

        taste_label = taste_label.float().to(device)
        taste_label = taste_label.unsqueeze(1)

        quantity_label = quantity_label.float().to(device)
        quantity_label = quantity_label.unsqueeze(1)

        delivery_label = delivery_label.float().to(device)
        delivery_label = delivery_label.unsqueeze(1)

        taste_out, quantity_out, delivery_out = model(input_ids, attention_mask_token, token_type_ids)

        # Loss
        ################################################################################################
        taste_loss = loss_fn(taste_out, taste_label)
        quantity_loss = loss_fn(quantity_out, quantity_label)
        delivery_loss = loss_fn(delivery_out, delivery_label)

        total_loss = (taste_loss + quantity_loss + delivery_loss) / 3
        total_loss.backward() 
        ################################################################################################

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # Accuracy
        ################################################################################################
        taste_preds = (taste_out >= 0.5).float()
        quantity_preds = (quantity_out >= 0.5).float()
        delivery_preds = (delivery_out >= 0.5).float()

        taste_accuracy = (taste_preds == taste_label).cpu().numpy().mean()
        quantity_accuracy = (quantity_preds == quantity_label).cpu().numpy().mean()
        delivery_accuracy = (delivery_preds == delivery_label).cpu().numpy().mean()

        train_acc += (taste_accuracy + quantity_accuracy + delivery_accuracy) / 3
        ################################################################################################

        # F1_Score
        ################################################################################################
        taste_f1 = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'micro')
        quantity_f1 = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'micro')
        delivery_f1 = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'micro')
        
        total_f1 +=  (taste_f1 + quantity_f1 + delivery_f1) / 3
        ################################################################################################

        if batch_id % log_interval == 0:

            print(f"epoch {e+1} batch id {batch_id+1} loss {total_loss.data.cpu().numpy():.3f} train acc {train_acc / (batch_id+1):.3f}")
            print(f'taste_accuracy {taste_accuracy:.3f}  | quantity_accuracy {quantity_accuracy:.3f}  | delivery_accuracy {delivery_accuracy:.3f}')
            print(f'f1_score:{total_f1 / (batch_id+1):.3f}')
            print('#'*80)

            # WandB
            #############################################################################################################################
            wandb.log({'interval_Loss': total_loss.data.cpu().numpy(),
                        'interval_taste_Accuracy' :taste_accuracy, 'interval_quantity_Accuracy' : quantity_accuracy, 'interval_delivery_Accuracy' : delivery_accuracy,
                        'interval_taste_F1' : taste_f1, 'interval_quantity_F1' : quantity_f1, 'interval_delivery_F1' : delivery_f1})
            #############################################################################################################################

    print(f"epoch {e+1} epoch_Train_Accuracy {train_acc / (batch_id+1):.3f}, epoch_F1_score {total_f1 / (batch_id+1):.3f}")
    
    # WandB
    #############################################################################################################################
    wandb.log({'epoch_Train_Accuracy' : train_acc / (batch_id+1), 'epoch_F1_score' : total_f1 / (batch_id+1)})
    #############################################################################################################################

    model.eval()
    test_f1 = 0.0
    test_acc = 0.0
    for batch_id, (input_ids, attention_mask_token, token_type_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(valid_dataloader)):
        
        input_ids = input_ids.long().to(device)
        input_ids = input_ids.squeeze(1)

        token_type_ids = token_type_ids.long().to(device)
        token_type_ids = token_type_ids.squeeze(1)
        
        attention_mask_token= attention_mask_token.squeeze(1)

        taste_label = taste_label.float().to(device)
        taste_label = taste_label.unsqueeze(1)

        quantity_label = quantity_label.float().to(device)
        quantity_label = quantity_label.unsqueeze(1)

        delivery_label = delivery_label.float().to(device)
        delivery_label = delivery_label.unsqueeze(1)

        taste_out, quantity_out, delivery_out = model(input_ids, attention_mask_token, token_type_ids)

        # Test_Accuracy
        ################################################################################################    
        taste_preds = (taste_out >= 0.5).float()
        quantity_preds = (quantity_out >= 0.5).float()
        delivery_preds = (delivery_out >= 0.5).float()

        taste_accuracy = (taste_preds == taste_label).cpu().numpy().mean()
        quantity_accuracy = (quantity_preds == quantity_label).cpu().numpy().mean()
        delivery_accuracy = (delivery_preds == delivery_label).cpu().numpy().mean()

        test_acc += (taste_accuracy + quantity_accuracy + delivery_accuracy) / 3
        ################################################################################################ 

        # Test_F1_Score
        ################################################################################################
        taste_f1 = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'micro')
        quantity_f1 = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'micro')
        delivery_f1 = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'micro')
        
        test_f1 +=  (taste_f1 + quantity_f1 + delivery_f1) / 3
        ################################################################################################

    print('-'*40,'Test_Value','-'*40)
    print(f"test_Accuracy {test_acc / (batch_id+1):.3f}, test_F1_score {test_f1/ (batch_id+1):.3f}")
    print('-'*92)

    # WandB
    #############################################################################################################################
    wandb.log({'Test_acc' : test_acc / (batch_id+1), 'Test_F1_Score' : test_f1/ (batch_id+1)})
    #############################################################################################################################

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/1380 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.721 train acc 0.427
taste_accuracy 0.469  | quantity_accuracy 0.344  | delivery_accuracy 0.469
f1_score:0.427
################################################################################
epoch 1 batch id 201 loss 0.228 train acc 0.727
taste_accuracy 0.906  | quantity_accuracy 0.938  | delivery_accuracy 1.000
f1_score:0.727
################################################################################
epoch 1 batch id 401 loss 0.165 train acc 0.829
taste_accuracy 1.000  | quantity_accuracy 0.875  | delivery_accuracy 0.969
f1_score:0.829
################################################################################
epoch 1 batch id 601 loss 0.122 train acc 0.868
taste_accuracy 0.875  | quantity_accuracy 1.000  | delivery_accuracy 0.969
f1_score:0.868
################################################################################
epoch 1 batch id 801 loss 0.245 train acc 0.887
taste_accuracy 0.875  | quantity_accuracy 0.938  | delivery_accuracy 0.938
f1_

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1380 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.129 train acc 0.969
taste_accuracy 1.000  | quantity_accuracy 0.969  | delivery_accuracy 0.938
f1_score:0.969
################################################################################
epoch 2 batch id 201 loss 0.101 train acc 0.958
taste_accuracy 0.938  | quantity_accuracy 0.969  | delivery_accuracy 0.969
f1_score:0.958
################################################################################
epoch 2 batch id 401 loss 0.054 train acc 0.957
taste_accuracy 0.969  | quantity_accuracy 1.000  | delivery_accuracy 1.000
f1_score:0.957
################################################################################
epoch 2 batch id 601 loss 0.085 train acc 0.957
taste_accuracy 0.969  | quantity_accuracy 0.969  | delivery_accuracy 1.000
f1_score:0.957
################################################################################
epoch 2 batch id 801 loss 0.126 train acc 0.957
taste_accuracy 0.969  | quantity_accuracy 0.969  | delivery_accuracy 0.906
f1_

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/34 [00:00<?, ?it/s]

---------------------------------------- Test_Value ----------------------------------------
test_Accuracy 0.908, test_F1_score 0.908
--------------------------------------------------------------------------------------------


In [23]:
wandb.finish()

Test_F1_Score,▁█
Test_acc,▁█
epoch_F1_score,▁█
epoch_Train_Accuracy,▁█
interval_Loss,█▃▂▂▃▂▃▂▁▁▁▂▂▃
interval_delivery_Accuracy,▁███▇█▇▇███▇██
interval_delivery_F1,▁███▇█▇▇███▇██
interval_quantity_Accuracy,▁▇▇█▇▇▇██████▇
interval_quantity_F1,▁▇▇█▇▇▇██████▇
interval_taste_Accuracy,▁▇█▆▆█▆█▇███▇█
interval_taste_F1,▁▇█▆▆█▆█▇███▇█


In [24]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/aiffel/AIFFELTON/model_save/element_klue_0920(29.84).pt')
# torch.save(model, '/content/drive/MyDrive/aiffel/AIFFELTON/model_save/pytorch_model.bin')

In [10]:
model = BERTClassifier(bertmodel, dr_rate=0.3).to(device)

In [11]:
model.load_state_dict(torch.load('/content/drive/MyDrive/aiffel/AIFFELTON/model_save/sweep_model_dict_0.913.pt', map_location=device))

<All keys matched successfully>

In [12]:
def predict(predict_sentence):

    dataset_another = pd.DataFrame({'review': predict_sentence, 'taste' : 0 , 'quantity' : 0, 'delivery' : 0}, index = [0])

    another_test = BERTDataset(dataset_another,tokenizer, max_len)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=1, num_workers=5)
    
    model.eval()

    for batch_id, (input_ids, attention_mask_token, token_type_ids, taste_label, quantity_label, delivery_label) in enumerate(test_dataloader):
            
        input_ids = input_ids.long().to(device)
        input_ids = input_ids.squeeze(1)

        token_type_ids = token_type_ids.long().to(device)
        token_type_ids = token_type_ids.squeeze(1)
        
        attention_mask_token= attention_mask_token.squeeze(1)

        taste_label = taste_label.long().to(device)
        quantity_label = quantity_label.long().to(device)
        delivery_label = delivery_label.long().to(device)

        taste_out, quantity_out, delivery_out = model(input_ids, attention_mask_token, token_type_ids)

        for taste, quantity, delivery in zip(taste_out, quantity_out, delivery_out):

            logits_taste = taste[0].detach().cpu().numpy()
            logits_quantity = quantity[0].detach().cpu().numpy()
            logits_delivery = delivery[0].detach().cpu().numpy()

            print(f'맛에 대한 점수: {logits_taste:.4f}')
            print(f'양 or 서비스에 대한 점수: {logits_quantity:.4f}')
            print(f'배달에 대한 점수: {logits_delivery:.4f}')

---
# 결과값 테스트
---

- epoch = 1

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
predict('맛이 없다')

맛에 대한 점수: 0.9833
양 or 서비스에 대한 점수: 0.0188
배달에 대한 점수: 0.0156


In [15]:
predict('서비스가 좋아서 다시 가고 싶어요')

맛에 대한 점수: 0.0912
양 or 서비스에 대한 점수: 0.9557
배달에 대한 점수: 0.0426


In [16]:
predict('돈가스 맛집이라해서 가봤는데 양이 너무 적어요')

맛에 대한 점수: 0.1625
양 or 서비스에 대한 점수: 0.9738
배달에 대한 점수: 0.0324


In [17]:
predict('음식은 겉바속촉 매장은 깔끔')

맛에 대한 점수: 0.5062
양 or 서비스에 대한 점수: 0.0069
배달에 대한 점수: 0.1211


In [18]:
predict('가까워서 빠른 배달이 좋아요 상하이 맛있구요')

맛에 대한 점수: 0.9599
양 or 서비스에 대한 점수: 0.0174
배달에 대한 점수: 0.9626


In [19]:
predict('맛 양 배달 모두 만족합니다')

맛에 대한 점수: 0.9109
양 or 서비스에 대한 점수: 0.9444
배달에 대한 점수: 0.9248


In [20]:
predict('생각보다 빨리 와서 좋았어요 ㅎㅎ')

맛에 대한 점수: 0.0268
양 or 서비스에 대한 점수: 0.0249
배달에 대한 점수: 0.9741


In [21]:
predict('다 좋은데 피자가 찌그러져서 왔네요')

맛에 대한 점수: 0.0274
양 or 서비스에 대한 점수: 0.0316
배달에 대한 점수: 0.9657


In [22]:
predict('사업 번창하세요')

맛에 대한 점수: 0.2553
양 or 서비스에 대한 점수: 0.0144
배달에 대한 점수: 0.0552


In [23]:
predict('콜라가 거의 다 넘쳤어요 조금 흐른 정도가 아니라 쏟아진 수준이라 감튀도 축축해지고')

맛에 대한 점수: 0.0291
양 or 서비스에 대한 점수: 0.0555
배달에 대한 점수: 0.9601


In [24]:
predict('기사님 너무 친절하시고 감자도 꾹꾹 담아서 가득 주셔서 배부르게 먹었어요 콜라도 뚜껑 이중으로 해주셔서 하나도 안 새고 왔습니당')

맛에 대한 점수: 0.0211
양 or 서비스에 대한 점수: 0.8510
배달에 대한 점수: 0.7185


In [25]:
predict('다른 집 들렸다 온 것 같아요')

맛에 대한 점수: 0.0259
양 or 서비스에 대한 점수: 0.0233
배달에 대한 점수: 0.8972


In [26]:
predict('배달 빠르고 맛있습니다')

맛에 대한 점수: 0.9506
양 or 서비스에 대한 점수: 0.0182
배달에 대한 점수: 0.9703


----


---
# inference
---

In [42]:
# !pip install soynlp

# from soynlp.normalizer import *
# import re 

In [43]:
# def extract_word(text):
#   text = text.lower() # 소문자 변환
#   first = emoticon_normalize(text, num_repeats = 2) # 이모티콘
#   first = repeat_normalize(first, num_repeats = 2) # ㅋㅋㅎㅎ 같은 반복 반복문장 2개만 남기고 

#   convert = re.compile('^\d*\d$|[^가-힣a-zA-Z0-9.ㅋㅜㅎㅠ]') # 해당 문자만 남기고 나머지 제거
#   result = convert.sub(' ',first) # 변환

#   result = re.sub(r'[" "]+', " ",result) # 공백 여러개를 한개로 변환
#   result = result.strip() # 양쪽 공백 제거

#   return result

In [27]:
modeling = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/3_element_dataset_modifying/Final_Test_Dataset_0920.csv')
del modeling['Unnamed: 0']
modeling['taste'] = 0
modeling['quantity'] = 0
modeling['delivery'] = 0

In [28]:
original = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/3_element_dataset_modifying/Final_Test_Dataset_0920.csv')
del original['Unnamed: 0']

In [29]:
another_test = BERTDataset(modeling,tokenizer, max_len)
test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=1, num_workers=5)

model.eval()

for batch_id, (input_ids, attention_mask_token, token_type_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm(test_dataloader)):
        
    input_ids = input_ids.long().to(device)
    input_ids = input_ids.squeeze(1)

    token_type_ids = token_type_ids.long().to(device)
    token_type_ids = token_type_ids.squeeze(1)
    
    attention_mask_token= attention_mask_token.squeeze(1)

    taste_label = taste_label.long().to(device)
    quantity_label = quantity_label.long().to(device)
    delivery_label = delivery_label.long().to(device)

    taste_out, quantity_out, delivery_out = model(input_ids, attention_mask_token, token_type_ids)

    for taste, quantity, delivery in zip(taste_out, quantity_out, delivery_out):

        logits_taste = taste.detach().cpu().numpy()
        logits_quantity = quantity.detach().cpu().numpy()
        logits_delivery = delivery.detach().cpu().numpy()

        if logits_taste >= 0.6:
          modeling['taste'][batch_id] = 1
        if logits_quantity >= 0.6:
          modeling['quantity'][batch_id] = 1
        if logits_delivery >= 0.6:
          modeling['delivery'][batch_id] = 1

100%|██████████| 1776/1776 [00:24<00:00, 72.80it/s]


----

In [30]:
modeling

,review,taste,quantity,delivery
0,배달 시간은 항상 지켜져서 정말 좋아요 맛도 있고 근데 고기 추가를 했는데 뭔가 양...,1,1,1
1,맛있었어요 담에 또 시켜먹을게욤,1,0,0
2,쏘쏘.....,0,0,0
3,방금 맛있게 한그릇 먹었습니다 배가 너무고파서 사진도 못찍었네요 재료의 신선도 맛 ...,1,1,0
4,배달 빨리 왔어요 따뜻한 피자 먹었네ㅎ 양도 많고 맛있게 잘 먹었습니다,1,1,1
...,...,...,...,...
1771,바삭바삭 항상 맛있어요,1,0,0
1772,음식 양을 많고 배송도 빨라요,0,1,1
1773,마늘맛 너무좋습니다 ㅜㅜ 창녕버거야 단종되지말고 오래오래 꽃길만걷자 배달도 엄청 빠르네요,1,0,1
1774,항상 가성비의 이모네 ㅎㅎ 간장 부탁드립니다 고등어는 ㅠ,0,0,0


In [30]:
inference = original.compare(modeling)
inference

taste       quantity       delivery      
      self other     self other     self other
2      0.0   1.0      NaN   NaN      NaN   NaN
5      0.0   1.0      NaN   NaN      NaN   NaN
10     0.0   1.0      NaN   NaN      NaN   NaN
12     0.0   1.0      NaN   NaN      NaN   NaN
13     0.0   1.0      NaN   NaN      NaN   NaN
...    ...   ...      ...   ...      ...   ...
1756   NaN   NaN      1.0   0.0      NaN   NaN
1763   NaN   NaN      1.0   0.0      NaN   NaN
1767   0.0   1.0      NaN   NaN      NaN   NaN
1774   NaN   NaN      1.0   0.0      NaN   NaN
1775   0.0   1.0      NaN   NaN      NaN   NaN

[470 rows x 6 columns]

In [31]:
error_rate = (len(inference) / len(original)) * 100
round(error_rate,2)

26.46

In [50]:
# modeling.to_csv('/content/drive/MyDrive/aiffel/AIFFELTON/data/append_0920.csv')